In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords #natural language toolkit where what a an the
from nltk.stem import PorterStemmer #stemming return root word by removing pre and post 
from sklearn.feature_extraction.text import TfidfVectorizer #text into feature extraction
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [3]:
#!pip install nltk

In [4]:
#printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### Data Pre-processing

In [5]:
#loading the dataset into pandas dataframe
news_dataset=pd.read_csv('train.csv')

In [6]:
news_dataset.shape

(20800, 5)

In [7]:
print(news_dataset.head())

   id                                              title              author  \
0   0  House Dem Aide: We Didn’t Even See Comey’s Let...       Darrell Lucus   
1   1  FLYNN: Hillary Clinton, Big Woman on Campus - ...     Daniel J. Flynn   
2   2                  Why the Truth Might Get You Fired  Consortiumnews.com   
3   3  15 Civilians Killed In Single US Airstrike Hav...     Jessica Purkiss   
4   4  Iranian woman jailed for fictional unpublished...      Howard Portnoy   

                                                text  label  
0  House Dem Aide: We Didn’t Even See Comey’s Let...      1  
1  Ever get the feeling your life circles the rou...      0  
2  Why the Truth Might Get You Fired October 29, ...      1  
3  Videos 15 Civilians Killed In Single US Airstr...      1  
4  Print \nAn Iranian woman has been sentenced to...      1  


In [8]:
#counting the number of missing values in the dataset
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
#replacing the null values with empty string

news_dataset=news_dataset.fillna('')
news_dataset.shape

(20800, 5)

In [10]:
#merging the auther name and news title
news_dataset['content']=news_dataset['author']+' '+news_dataset['title']

In [11]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [12]:
X=news_dataset.drop(columns='label',axis=1)
Y=news_dataset['label']

In [13]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Stemming:
Stemming is the process of reducing a word to its Root word
ex. actor,actress, acting.....>act

In [14]:
port_stem=PorterStemmer() #root word

In [15]:
def stemming(content):
    stemmed_content=re.sub('{^a-zA-Z}',' ',content) #remove punct and space numbers return characters regular expression
    stemmed_content=stemmed_content.lower()
    stemmed_content=stemmed_content.split() #make a list of words
    stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] #remove root word
    stemmed_content=' '.join(stemmed_content)
    return stemmed_content

In [16]:
news_dataset['content']=news_dataset['content'].apply(stemming)

In [17]:
print(news_dataset['content'])

0        darrel lucu hous dem aide: didn’t even see com...
1        daniel j. flynn flynn: hillari clinton, big wo...
2                  consortiumnews.com truth might get fire
3        jessica purkiss 15 civilian kill singl us airs...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper t.i.: trump ’poster child ...
20796    benjamin hoffman n.f.l. playoffs: schedule, ma...
20797    michael j. de la merc rachel abram macy’ said ...
20798    alex ansari nato, russia hold parallel exercis...
20799                         david swanson keep f-35 aliv
Name: content, Length: 20800, dtype: object


In [18]:
#seperating the data and label
X=news_dataset['content'].values
Y=news_dataset['label'].values

In [19]:
print(X)

['darrel lucu hous dem aide: didn’t even see comey’ letter jason chaffetz tweet'
 'daniel j. flynn flynn: hillari clinton, big woman campu - breitbart'
 'consortiumnews.com truth might get fire' ...
 'michael j. de la merc rachel abram macy’ said receiv takeov approach hudson’ bay - new york time'
 'alex ansari nato, russia hold parallel exercis balkan'
 'david swanson keep f-35 aliv']


In [20]:
print(Y)

[1 0 1 ... 0 1 1]


In [21]:
Y.shape

(20800,)

In [27]:
#converting the textual data to numerical data
vectorizer=TfidfVectorizer()
vectorizer.fit(X)
X=vectorizer.transform(X)
#frequency of words gives some value

In [28]:
print(X)

  (0, 19097)	0.27315635150958634
  (0, 16473)	0.23676064517956458
  (0, 11072)	0.33384522056560495
  (0, 10747)	0.2682220926318631
  (0, 9692)	0.22757176689298136
  (0, 8832)	0.205341824533184
  (0, 6433)	0.2142258791026174
  (0, 5256)	0.27468869329117757
  (0, 4995)	0.2512923264945339
  (0, 4763)	0.3304457115379666
  (0, 3952)	0.22664699692052692
  (0, 3403)	0.3375689613898566
  (0, 809)	0.3646500188253279
  (1, 20416)	0.29951908908156866
  (1, 8608)	0.19815023888659125
  (1, 7101)	0.711483310803025
  (1, 4728)	0.26268668599849243
  (1, 3778)	0.19062686807106288
  (1, 3100)	0.3870784468942128
  (1, 2713)	0.15460118725006144
  (1, 2258)	0.2928176012009572
  (2, 19015)	0.41491113753784553
  (2, 11878)	0.49151393723208897
  (2, 7650)	0.34605253138342823
  (2, 6968)	0.39293503470255664
  :	:
  (20797, 15142)	0.2718547014681534
  (20797, 14906)	0.24707379158386517
  (20797, 12685)	0.08013411384942215
  (20797, 11844)	0.17405416749037161
  (20797, 11767)	0.2945753459972408
  (20797, 11179)	

In [29]:
#splitting dataset to training and test data

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,stratify=Y,test_size=0.2,random_state=2)

#### Training the model: Logistic Regression model

In [30]:
model=LogisticRegression()

In [31]:
model.fit(X_train,Y_train)

LogisticRegression()

#### Accuracy Score

In [32]:
# accuracy score on training data

X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)
print(training_data_accuracy)

0.98671875


In [33]:
# accuracy score on testing data

X_test_prediction=model.predict(X_test)
testing_data_accuracy=accuracy_score(X_test_prediction,Y_test)
print(testing_data_accuracy)

0.9776442307692308


### Making predictive system

In [34]:
X_new=X_test[0]
prediction=model.predict(X_new)

if prediction[0]==0:
    print("Real news")
else:
    print("Fake news")

Fake news
